[View in Colaboratory](https://colab.research.google.com/github/jefftflearn/py3-metmuseum-crawel/blob/master/metmuseumCrawel.ipynb)

In [0]:
!pip install beautifulsoup4

pip 10.0.1 from /usr/local/lib/python3.6/dist-packages/pip (python 3.6)


In [0]:
!pip install -U -q PyDrive
import urllib
import ssl
from bs4 import BeautifulSoup
from time import sleep
import os
import json
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def readHtmlSourceCode(url):
  response = urllib.request.urlopen(url)
  srcCode = response.read()
  return srcCode

def downloadImageFromUrlThenUpload(driver, imgUrl, title):
  fname = title.replace(" ", "_").replace(";", "#").replace(",", "##").replace("(", "$").replace(")", "$")
  path = './' + title.replace(" ", "_").replace(";", "#").replace(",", "##").replace("(", "$").replace(")", "$")
  try:
    response = urllib.request.urlopen(imgUrl)
    file = open(path, "wb")
    file.write(response.read())
    file.close()
    uploaded = drive.CreateFile({'title': fname})
    #uploaded.SetContentString(response.read())
    uploaded.SetContentFile(path)
    uploaded.Upload()
    print('upload success '+fname);
    os.remove(path)
    print('remove temp file completed')
  except urllib.error.HTTPError as err:
    print("404 not found")
  
def uploadCsvFile(driver, str):
  uploaded = drive.CreateFile({'title': 'metmuseum_'+q+".txt"})
  uploaded.SetContentString(str)
  uploaded.Upload()
  print("upload db index file success")
  
ssl._create_default_https_context=ssl._create_unverified_context
  
keywords = 'Generative Adversarial Nets'
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print('接入google drive')

token = 'https://images.metmuseum.org/CRDImages/'
perPage = 20
offset = 1

q = 'Raffaello%20Sanzio'
i = 0
count = 0
csvStr = ''
while count < 9:
  offset = perPage * i + 1
  url = 'https://www.metmuseum.org/api/collection/collectionlisting?artist=&department=&era=&geolocation=&material=&offset='+str(offset)+'&pageSize=0&perPage='+str(perPage)+'&q='+q+'&showOnly=&sortBy=Relevance&sortOrder=asc'

  html = readHtmlSourceCode(url)
  jsObj = json.loads(html)
  dataset = jsObj['results']
  request = jsObj['request']
  i += 1
  if len(dataset) != 0 :
    
    print("==============================================================================================================offset: "+str(offset))

    for item in dataset[:] :
      orginImage = item['largeImage'].replace('web-large', 'original')
      orginImageUrl = token + orginImage
      if 'noimage' not in orginImageUrl:
        print(item)
        medium = item['medium']
        description = item['description']
        title = item['title']
        downloadImageFromUrlThenUpload(drive, orginImageUrl, title)
        csvStr += title + " $$$$ " + description + " $$$$ " + medium + " $$$$ " + orginImageUrl + "\r\n"
        #print(orginImageUrl)
  else:
    break
  count += 1
uploadCsvFile(drive, csvStr)
print(count)
    


